In [80]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from feature_vector import get_feature_names
from Utils.feature_ranking import load_rankSelections
import pandas as pd
import numpy as np



from setupDataset import get_dataset, combine_datasets

m_dataset = get_dataset("dataset.pickle")
w_dataset = get_dataset("wiki_dataset.pickle")

c_dataset = combine_datasets(m_dataset, w_dataset)

X, y = m_dataset["data"], m_dataset["target"]
#X, y = w_dataset["data"], w_dataset["target"]
#X, y = c_dataset["data"], c_dataset["target"]

estimator = LogisticRegression()
selector = RFE(estimator, 52, step=1)
selector = selector.fit(X, y)
sel = selector.get_support(indices=True)
sel

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 14, 17, 22, 23, 24, 25,
       26, 27, 28, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45,
       46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 58, 59, 61, 62, 63, 64, 66,
       67])

In [41]:
type(sel)

numpy.ndarray

Ok, was ist zu tun, was will ich?
Ich moechte gerne ein RFE-Feature Selection Ranking haben für alle Datensets, also genauso wie ich es bei den univariaten habe. von mir aus in einer separaten File, wichtig ist mir einfach die Reihung der Indices.

Plan, zuerst einmal fuer das kleinste Datenset durchfuehren und dann fuer die beiden weiteren Sets ebenso.

Wie also vorgehen?



In [83]:
def getRFE_ranking(dataset, out):
    X = dataset["data"]
    y = dataset["target"]
    
    n = len(get_feature_names())+1
    
    first = getRFE_selection(X, y, 1)
    RFE_ranks = [first[0]]
   
    for i in range(2, n):
        sel = getRFE_selection(X,y,i)
        diff = list(set(RFE_ranks).symmetric_difference(sel))
        
        for d in diff:
            if d not in RFE_ranks:  # apparently the ranking is not always identical, prevents an index from being added twice     
                RFE_ranks.append(d)
        

    RFE_ranks = np.array(RFE_ranks)
    save_RFE_ranks(RFE_ranks, out)
    
    return RFE_ranks
        

def save_RFE_ranks(ranks, out):
    col_label = ["RFE"]
    df = pd.DataFrame(ranks, columns=col_label)
    df.to_csv(out, sep='\t')
    

def getRFE_selection(X, y, n):
    estimator = LogisticRegression()
    selector = RFE(estimator, n, step=1)
    selector = selector.fit(X, y)
    selection = selector.get_support(indices=True)
    return selection.tolist()

In [94]:
m_RFE = getRFE_ranking(c_dataset, 'c_RFE_ranks.csv')
print "hello"
m_RFE

hello


array([24, 56, 62, 61, 64, 59, 60, 23,  0, 33, 47, 28, 29, 32,  5,  4, 30,
       40, 44, 52, 22, 67, 45, 15, 17, 35, 31, 66, 38,  7, 27,  6, 34, 58,
       51,  3, 50, 53,  2, 49, 11, 54, 55, 36, 57,  9,  1, 16, 42, 10, 14,
       39, 37, 46, 26, 41, 48, 65, 19, 21, 18,  8, 13, 25, 12, 43, 20, 63])

In [96]:
r = load_rankSelections('c_RFE_ranks.csv')
r2 = load_rankSelections('rankSelections_myDataset.csv')
np.concatenate((r2, r), axis=1)

array([[ 1,  1, 43, 60,  1, 24],
       [43, 43, 10,  1, 10, 56],
       [40, 10,  1, 56, 43, 62],
       [10, 26, 18, 10, 40, 61],
       [59, 11, 20, 59, 56, 64],
       [11, 35, 19, 24, 19, 59],
       [26, 12, 40, 12, 20, 60],
       [35, 59, 34, 61, 18, 23],
       [56, 19,  8, 43, 12,  0],
       [24, 62, 16, 19, 59, 33],
       [60, 24,  3, 20, 26, 47],
       [62, 20, 56, 40, 61, 28],
       [ 0, 18, 46, 62, 60, 29],
       [61, 60,  2, 46, 24, 32],
       [ 2, 40,  7, 26, 62,  5],
       [12, 56, 47, 11, 11,  4],
       [18,  2, 54, 18, 34, 30],
       [34, 52, 21, 23, 35, 40],
       [19, 23, 42, 34,  0, 44],
       [20, 61,  0, 45, 46, 52],
       [46,  9, 12, 35, 52, 22],
       [53,  0, 53, 64,  3, 67],
       [58, 21, 38, 16, 47, 45],
       [52, 55, 61, 52,  2, 15],
       [54, 51, 36, 66, 16, 17],
       [47, 13, 55, 51, 54, 35],
       [42, 33, 66, 58,  8, 31],
       [ 3, 50, 45, 33, 23, 66],
       [23, 48,  9, 13,  7, 38],
       [ 4, 32, 17, 31,  9,  7],
       [63

In [34]:
list(set(range(0,68)).symmetric_difference(m_RFE))

[58, 60, 63, 65, 66]